In [130]:
# Required packages
library(MASS)
library(mvtnorm)
library(polycor)

In [133]:
# Read the original data on Github
df_ori = read.csv(file="https://raw.githubusercontent.com/solmazahmadi/Computational-Statistics-Project/master/Data/Data_Original.csv")
df = df_ori[,-14]

In [134]:
# Make factors as factors and numeric as numeric

#Factors
df$CHAS = as.factor(df$CHAS)
df$ZN = as.factor(df$ZN)
df$RAD = as.factor(df$RAD)

#Numeric
df$CRIM = as.numeric(as.character(df$CRIM))
df$INDUS = as.numeric(as.character(df$INDUS))
df$NOX = as.numeric(as.character(df$NOX))
df$RM = as.numeric(as.character(df$RM))
df$AGE = as.numeric(as.character(df$AGE))
df$DIS = as.numeric(as.character(df$DIS))
df$TAX = as.numeric(as.character(df$TAX))
df$PTRATIO = as.numeric(as.character(df$PTRATIO))
df$B = as.numeric(as.character(df$B))
df$LSTAT = as.numeric(as.character(df$LSTAT))

In [137]:
set.seed(13)
SimulateData <- function(dataset, digits=2, n=NA, 
  use.names=TRUE, use.levels=TRUE, use.miss=TRUE,
  mvt.method="eigen", het.ML=FALSE, het.suppress=TRUE){

  # This function takes as argument an existing dataset, which 
  # must be either a matrix or a data frame. Each column of the 
  # dataset must consist either of numeric variables or ordered 
  # factors. When one or more ordered factors are included, 
  # then a heterogeneous correlation matrix is computed using 
  # John Fox' polycor package. Pairwise complete observations 
  # are used for all covariances, and the exact pattern of 
  # missing data present in the input is placed in the output,
  # provided a new sample size is not requested. Warnings from
  # the hetcor function are suppressed.

  # Author:   Ryne Estabrook
  # Created:  17 Aug 2010

  require(mvtnorm)
  require(polycor)

  # requires data frame or matrix
  if((is.data.frame(dataset)+is.matrix(dataset))==0){
    warning("Data must be a data frame or matrix")
  }

  # organization
  row <- dim(dataset)[1] # number of rows
  if(is.na(n))(n <- row) # sets unspecified sample size to num rows
  col <- dim(dataset)[2] # number of columns
  del <- is.na(dataset)  # records position of NAs in dataset
  if(n!=row){
    select <- round(runif(n, 0.5, row+.49),0)
    del <- del[select,]
  }
  num <- rep(NA, col)    # see what's not a factor
  ord <- rep(NA, col)    # see what's an ordered factor

  # which columns are numeric (the others are factors)?
  for (i in 1:col){
    num[i] <- is.numeric(dataset[,i])
    ord[i] <- is.ordered(dataset[,i])
  }

  # check for unordered factors
  location <- !(num|ord)
  unorder <- sum(location)

  if(unorder>0)warning(
    paste("Unordered factor detected in variable(s):", 
      names(dataset)[location]
    )
  )


  # if there are factors, we start here

  # find the variable means (constrain to zero for factors)
  mixedMeans <- rep(0, col)
  mixedMeans[num] <- apply(dataset[,num], 2, mean, na.rm=TRUE)

  # estimate a heterogeneous correlation matrix
  if (het.suppress==TRUE){
    suppressWarnings(het <- hetcor(dataset, ML=het.ML))
  } else (het <- hetcor(dataset, ML=het.ML))
  mixedCov <- het$correlations

  # make a diagonal matrix of standard deviations to turn the 
  # correlation matrix into a covariance matrix
  stand <- matrix(0, col, col)
  diag(stand) <- rep(1, col)
  diag(stand)[num] <- apply(dataset[,num], 2, sd, na.rm=TRUE)
  # pre and post multiply hetero cor matrix by diagonal sd matrix
  mixedCov <- stand %*% mixedCov %*% stand

  # generate the data
  simulate <- as.data.frame(rmvnorm(row, mixedMeans, mixedCov, mvt.method))

  # insert the missing data, if so requested
  if(use.miss==TRUE)(simulate[del] <- NA)

  # turn the required continuous variables into factors
  for (i in (1:col)[!num]){
    # the original data for this column
    old <- dataset[,i]
   
    # the new data for this column, omiting NAs
    new <- simulate[!is.na(simulate[,i]),i]

    # what are the levels of the original factor?
    lev <- levels(old)

    # establish cutpoints in new variable from cdf of old factor
    cut <- cumsum(table(old))/(sum(!is.na(old)))

    # put continuous variable into a matrix, repeating value across columns
    wide <- matrix(new, length(new), length(lev))

    # put the cutpoints in a matrix, repeating the cut point values across rows
    crit <- matrix(quantile(new, cut), length(new), length(lev), byrow=TRUE)

    # for each value (row of the wide matrix), 
    # how many cutpoints is the value greater than?
    # number of cutpoints surpassed=category
    simulate[!is.na(simulate[,i]),i] <- apply(wide>crit, 1, sum)

    # make it a factor
    simulate[,i] <- factor(simulate[,i], ordered=TRUE)

    # give the new factor the same levels as the old variable
    if(length(levels(simulate[,i]))!=length(lev))message(
      paste("Fewer categories in simulated variable", 
      names(simulate)[i], "than in input variable", names(dataset)[i]))
    if(use.levels==TRUE&(length(levels(simulate[,i]))==length(lev))){
      levels(simulate[,i]) <- lev} else (levels(simulate[,i]) <- 1:length(lev))
  }

  # round the data to the requested digits
  simulate[,num] <- round(simulate[,num], digits)

  # give the variables names, if so requested
  if(use.names==TRUE)(names(simulate) <- names(dataset))
  
  # return the new data
  return(simulate)
}

In [138]:
SimulateData(df)

Warning message in SimulateData(df):
"Unordered factor detected in variable(s): ZNUnordered factor detected in variable(s): CHASUnordered factor detected in variable(s): RAD"

CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
11.30,0,25.08,0,0.76,6.60,111.77,0.78,24,598.56,17.43,362.79,12.42
-7.97,0,12.96,1,0.68,6.28,87.22,3.14,24,450.92,16.58,402.21,13.69
-9.01,0,3.37,0,0.58,7.27,48.22,4.63,3,296.18,17.10,357.40,3.75
-2.20,0,19.14,0,0.54,5.47,58.88,2.37,3,445.28,21.51,374.54,16.78
5.63,0,18.21,0,0.67,6.17,114.02,2.33,4,500.39,19.11,265.51,23.87
-6.75,0,15.30,0,0.56,5.87,64.21,2.97,2,377.02,16.18,507.85,8.41
1.24,0,8.38,0,0.47,5.10,88.22,3.55,4,223.58,20.71,466.93,14.11
2.39,0,6.09,0,0.61,5.60,72.14,4.31,5,386.64,19.78,225.12,14.79
-2.30,0,11.04,0,0.46,6.92,40.31,6.98,3,270.96,17.79,369.93,12.75
8.89,0,10.23,0,0.55,6.67,67.17,1.99,8,396.10,17.72,273.17,8.66
